<a href="https://colab.research.google.com/github/savancs/product_recommender/blob/main/Basic_DistilBERT_DLAIEF23_PR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports & Setup - Check GPU!

## Packages/Libraries

In [3]:
!pip install gradio
!pip install transformers
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.6/304.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 6.9 MB/s

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import gradio
import torch

from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import wandb

from torch.utils.data import DataLoader, TensorDataset

In [ ]:
!wandb login --relogin 59add063e6bbf968685833105867f8be02a2191a

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## Data

In [ ]:
# product data

sephora_data = 'https://raw.githubusercontent.com/torrileigh/sephora_cleaned_11.26.23/main/cosmetic_p11.26.23.csv'
sephora_df = pd.read_csv(sephora_data)

sephora_df.head()

,Label,brand,name,price,rank,ingredients,Combination,Dry,Normal,Oily,Sensitive
0,Moisturizer,ALGENIST,GENIUS Ultimate Anti-Aging Cream,$112.00,4.3,"Water, Caprylic/Capric Triglyceride, Hydrogent...",1,1,1,1,1
1,Moisturizer,ALGENIST,Overnight Restorative Cream,$94.00,4.4,"Water, Isopropyl Isostearate, Butyrospermum Pa...",0,0,0,0,0
2,Moisturizer,ALGENIST,SUBLIME DEFENSE Ultra Lightweight UV Defense F...,$28.00,4.4,"Cyclopentasiloxane, Water, Alcohol Denat., Gly...",0,0,0,0,0
3,Moisturizer,ALGENIST,POWER Recharging Night Pressed Serum,$95.00,4.4,"Cocos Nucifera (Coconut) Water, Water (Aqua, E...",1,1,1,1,1
4,Moisturizer,ALGENIST,Firming & Lifting Neck Cream,$98.00,3.9,"Water, Isononyl Isononanoate, Dimethicone, But...",0,0,0,0,0


In [ ]:
# ingredient mapping data for skin concerns

ingredient_mapping_data = 'https://raw.githubusercontent.com/savancs/product_recommender/tori_branch/PR%20Text%20Input%20Dataset%20-%20Ingredient%20Mapping.csv'
ingredient_mapping_df = pd.read_csv(ingredient_mapping_data)

ingredient_mapping_df.head()

,Skin Concern,Preferred Ingredients,Excluded Ingredients
0,Sensitive,"Aloe Vera, Argan Oil, Borage Oil, Calendula, C...","Synthetic Fragrance, Essential Oils, Methyliso..."
1,Acne,"Salicylic Acid, Benzoyl Peroxide, Hyaluronic A...","Cocoa Butter, Lanolin, Mineral Oil, Paraffin, ..."
2,Oily,"Salicylic Acid, Niacinamide, Vitamin B3, Honey...","Coconut Oil, Cocoa Butter, Mineral Oil, Silico..."
3,Dry,"Hyaluronic acid, Glycerin, Ceramides, Shea But...","Alcohol, Fragrance, Salicylic Acid, Benzoyl Pe..."
4,Combination,"Hyaluronic acid, Ceramides, Lactic Acid, Antio...","Alcohol, Fragrance, Essential Oils, Benzoyl Pe..."


In [ ]:
#we aren't using this now, may need later.  Keeping just in case

test_val_data = 'https://raw.githubusercontent.com/savancs/product_recommender/tori_branch/GPT%20Dataset%20PR.csv'
test_val_df = pd.read_csv(test_val_data)

test_val_df.head()

,Input Text,Product Type,Min Price,Max Price,Skin Concerns,Preferred Ingredients,Excluded Ingredients
0,Looking for a sun protect for dry skin $36-$70...,Sun Protect,36.0,70.0,Dry,Peptides,NaN
1,Looking for a eye cream for combination skin $...,Eye Cream,18.0,40.0,Combination,NaN,Phthalates
2,Looking for a moisturizer for dry skin $40-$72...,Moisturizer,40.0,72.0,Dry,NaN,Formaldehyde
3,Looking for a treatment for dry skin $20-$40 w...,Treatment,20.0,40.0,Dry,NaN,Parabens
4,Looking for a face mask for acne-prone skin $3...,Face Mask,37.0,44.0,Acne,Aloe Vera,NaN


In [ ]:
# our complete annotations dataset

import json
import requests

url = 'https://raw.githubusercontent.com/savancs/product_recommender/tori_branch/annotations-test%20combo.json'

response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    annotated_df = json.loads(response.content)
else:
    print("Failed to download the data")


In [ ]:
print(annotated_df.keys())

dict_keys(['classes', 'annotations'])


In [ ]:
ner_annotations = annotated_df['annotations']
ner_classes = annotated_df['classes']

# checking it loaded well
for i, entry in enumerate(ner_annotations[:10]):
    if entry is not None:
        text, annotation_details = entry
        print(f"Entry {i+1}:")
        print("Text:", text)
        print("Annotations:", annotation_details)
        print("\n")
    else:
        print(f"Entry {i+1}: None\n")


Entry 1: None

Entry 2:
Text: Combination skin eye cream more than $6 but less than $16
Annotations: {'entities': [[0, 11, 'B-SKIN_CONCERN'], [12, 16, 'O'], [17, 20, 'B-PRODUCT_TYPE'], [21, 26, 'I-PRODUCT_TYPE'], [27, 31, 'O'], [32, 36, 'O'], [37, 39, 'B-MIN_PRICE'], [40, 43, 'O'], [44, 48, 'O'], [49, 53, 'O'], [54, 57, 'B-MAX_PRICE']]}


Entry 3:
Text: Cleanser with seaweed more than $41 but less than $95
Annotations: {'entities': [[0, 8, 'B-PRODUCT_TYPE'], [9, 13, 'O'], [14, 21, 'B-PREF_INGREDIENT'], [22, 26, 'O'], [27, 31, 'O'], [32, 35, 'B-MIN_PRICE'], [36, 39, 'O'], [40, 44, 'O'], [45, 49, 'O'], [50, 53, 'B-MAX_PRICE']]}


Entry 4:
Text: What's a good eye cream to use for skin that's sensitive?
Annotations: {'entities': [[0, 4, 'O'], [4, 6, 'O'], [7, 8, 'O'], [9, 13, 'O'], [14, 17, 'B-PRODUCT_TYPE'], [18, 23, 'I-PRODUCT_TYPE'], [24, 26, 'O'], [27, 30, 'O'], [31, 34, 'O'], [35, 39, 'O'], [40, 46, 'O'], [47, 56, 'B-SKIN_CONCERN'], [56, 57, 'O']]}


Entry 5:
Text: I need a face mask 

In [ ]:
ner_annotations = annotated_df['annotations']
ner_classes = annotated_df['classes']

start_index = -10 if len(ner_annotations) >= 10 else -len(ner_annotations)

# Print the last 10 entries
for i, entry in enumerate(ner_annotations[start_index:], start=start_index + len(ner_annotations) + 1):
    if entry is not None:
        text, annotation_details = entry
        print(f"Entry {i}:")
        print("Text:", text)
        print("Annotations:", annotation_details)
        print("\n")
    else:
        print(f"Entry {i}: None\n")

Entry 292:
Text: Looking for a face mask for oily skin $35-$48
Annotations: {'entities': [[0, 7, 'O'], [8, 11, 'O'], [12, 13, 'O'], [14, 18, 'B-PRODUCT_TYPE'], [19, 23, 'I-PRODUCT_TYPE'], [24, 27, 'O'], [28, 32, 'B-SKIN_CONCERN'], [33, 37, 'O'], [38, 42, 'B-MIN_PRICE'], [42, 45, 'B-MAX_PRICE']]}


Entry 293:
Text: Looking for a cleanser for acne-prone skin $23-$52
Annotations: {'entities': [[0, 7, 'O'], [8, 11, 'O'], [12, 13, 'O'], [14, 22, 'B-PRODUCT_TYPE'], [23, 26, 'O'], [27, 37, 'B-SKIN_CONCERN'], [38, 42, 'O'], [43, 47, 'B-MIN_PRICE'], [47, 50, 'B-MAX_PRICE']]}


Entry 294:
Text: Looking for a face mask for combination skin $49-$67
Annotations: {'entities': [[0, 7, 'O'], [8, 11, 'O'], [12, 13, 'O'], [14, 18, 'B-PRODUCT_TYPE'], [19, 23, 'I-PRODUCT_TYPE'], [24, 27, 'O'], [28, 39, 'B-SKIN_CONCERN'], [40, 44, 'O'], [45, 49, 'B-MIN_PRICE'], [49, 52, 'B-MAX_PRICE']]}


Entry 295:
Text: Looking for a face mask for sensitive skin $37-$77 with Ceramides without Phthalates
Annotations: {'en

In [ ]:
#Checking structure
print(ner_classes)

['B-PRODUCT_TYPE', 'I-PRODUCT_TYPE', 'B-SKIN_CONCERN', 'B-PREF_INGREDIENT', 'I-PREF_INGREDIENT', 'B-MIN_PRICE', 'B-MAX_PRICE', 'B-EXCL_INGREDIENT', 'I-EXCL_INGREDIENT', 'O']


In [ ]:
# Writing a function to split our annotations data into trainig, testing, validation
# split is roughly 70%/15%/15%

def split_annotations(data, train_size=210, test_size=45, val_size=45):
    """
    Splits the annotations data into training, testing, and validation sets.

    :param data: List of annotations.
    :param train_size: Size of the training set.
    :param test_size: Size of the test set.
    :param val_size: Size of the validation set.
    :return: Three lists representing the train, test, and validation sets.
    """
    total_length = len(data)
    all_indices = list(range(total_length))

    # Create evenly spaced indices for test and validation sets
    interval = total_length // (test_size + val_size)
    test_indices = all_indices[0:total_length:interval][:test_size]
    val_indices = all_indices[1:total_length:interval][:val_size]

    # Remaining indices for training set
    train_indices = list(set(all_indices) - set(test_indices) - set(val_indices))
    train_set = [data[i] for i in train_indices[:train_size]]

    # Create test and validation sets
    test_set = [data[i] for i in test_indices]
    val_set = [data[i] for i in val_indices]

    return train_set, test_set, val_set


In [ ]:
# This actually splits 'annotations' data
train_df, test_df, val_df = split_annotations(annotated_df['annotations'])

# Check sizes of each df
len(train_df), len(test_df), len(val_df)

(210, 45, 45)

## Save dfs to Drive

Added in the two cells below to save this data in drive and be able to call it in other notebooks for testing/validation.

File folders must be created ahead of time within your google drive to do this and file names must match.  This isn't necessary to run.  I was experimenting with running stuff in a second notebook, but it ended up being more trouble than it was worth.


(Don't need to run second, do need first)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import json

# Function to save our split datasets retaining JSON filetype
def save_json(data, file_path):
    with open(file_path, 'w') as file:
        json.dump(data, file)

# Specify paths to save in Google Drive
train_path = '/content/drive/My Drive/DLAIEF23_Data/train_data.json'
test_path = '/content/drive/My Drive/DLAIEF23_Data/test_data.json'
val_path = '/content/drive/My Drive/DLAIEF23_Data/val_data.json'

# Call function & save
save_json(train_df, train_path)
save_json(test_df, test_path)
save_json(val_df, val_path)


# Model Main

Reminder to self of important names:

sephora_df

test_val_df

train_df

## NER - Use this Section!

In [ ]:
# Process each annotated text in ner_annotations to align tokens with their labels
from transformers import AutoTokenizer

# Initialize the tokenizer
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Define the global counter for None values (should end up discarding 3)
none_counter = 0

# Define the process_annotation function
def process_annotation(annotation):
    global none_counter

    if annotation is None:
        none_counter += 1
        return None

    content = annotation[0]
    entities = annotation[1]['entities']
    tokenized_input = tokenizer(content, truncation=True, padding='max_length', max_length=512)
    tokens = tokenizer.convert_ids_to_tokens(tokenized_input.input_ids)

    # Initialize labels for each token as 'O'
    aligned_labels = ['O'] * len(tokenized_input['input_ids'])

    # Align labels with BERT tokens
    for start, end, label in entities:
        for i in range(start, end + 1):
            if i < len(tokenized_input['input_ids']):
                prefix = 'B-' if i == start else 'I-'
                aligned_labels[i] = prefix + label

    return tokenized_input['input_ids'], tokenized_input['attention_mask'], aligned_labels


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Process all annotations, excluding None values entirely
processed_data = [process_annotation(annotation) for annotation in train_df if annotation is not None]

# Print the number of None values discarded
print(f"Number of None values discarded: {none_counter}")

Number of None values discarded: 0


In [ ]:
# function to correct this wonky thing that happens with the labels
# some extra garbage is added to the front, this strips that off

def correct_labels(labels):
    corrected_labels = []
    for label_seq in labels:
        corrected_seq = []
        for label in label_seq:

            # Handle the 'O' label correctly
            if label == 'O' or label == 'B-O' or label == 'I-O':
                corrected_label = 'O'

            else:
                parts = label.split('-')
                if len(parts) > 2:
                    # Remove the redundant prefix
                    corrected_label = '-'.join(parts[-2:])

                else:
                    corrected_label = label
            corrected_seq.append(corrected_label)
        corrected_labels.append(corrected_seq)
    return corrected_labels

In [ ]:
#double check we've dropped our none's
for item in processed_data:
    if item is None or not isinstance(item, tuple) or len(item) != 3:
        print("Found an issue with processed data:", item)

In [ ]:
# Function to convert labels to indices
def convert_labels_to_indices(labels, label_map):
    return [[label_map[label] for label in label_seq] for label_seq in labels]

In [ ]:
# Extract labels from processed annotations
extracted_labels = [aligned_labels for _, _, aligned_labels in processed_data if aligned_labels is not None]
print(extracted_labels)

[['B-B-PRODUCT_TYPE', 'I-B-PRODUCT_TYPE', 'I-B-PRODUCT_TYPE', 'I-B-PRODUCT_TYPE', 'I-B-PRODUCT_TYPE', 'I-B-PRODUCT_TYPE', 'I-B-PRODUCT_TYPE', 'I-B-PRODUCT_TYPE', 'I-B-PRODUCT_TYPE', 'B-O', 'I-O', 'I-O', 'I-O', 'I-O', 'B-B-PREF_INGREDIENT', 'I-B-PREF_INGREDIENT', 'I-B-PREF_INGREDIENT', 'I-B-PREF_INGREDIENT', 'I-B-PREF_INGREDIENT', 'I-B-PREF_INGREDIENT', 'I-B-PREF_INGREDIENT', 'I-B-PREF_INGREDIENT', 'B-O', 'I-O', 'I-O', 'I-O', 'I-O', 'B-O', 'I-O', 'I-O', 'I-O', 'I-O', 'B-B-MIN_PRICE', 'I-B-MIN_PRICE', 'I-B-MIN_PRICE', 'I-B-MIN_PRICE', 'B-O', 'I-O', 'I-O', 'I-O', 'B-O', 'I-O', 'I-O', 'I-O', 'I-O', 'B-O', 'I-O', 'I-O', 'I-O', 'I-O', 'B-B-MAX_PRICE', 'I-B-MAX_PRICE', 'I-B-MAX_PRICE', 'I-B-MAX_PRICE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

In [ ]:
# Correct the labels
corrected_labels = correct_labels(extracted_labels)
print(corrected_labels)

[['B-PRODUCT_TYPE', 'B-PRODUCT_TYPE', 'B-PRODUCT_TYPE', 'B-PRODUCT_TYPE', 'B-PRODUCT_TYPE', 'B-PRODUCT_TYPE', 'B-PRODUCT_TYPE', 'B-PRODUCT_TYPE', 'B-PRODUCT_TYPE', 'O', 'O', 'O', 'O', 'O', 'B-PREF_INGREDIENT', 'B-PREF_INGREDIENT', 'B-PREF_INGREDIENT', 'B-PREF_INGREDIENT', 'B-PREF_INGREDIENT', 'B-PREF_INGREDIENT', 'B-PREF_INGREDIENT', 'B-PREF_INGREDIENT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MIN_PRICE', 'B-MIN_PRICE', 'B-MIN_PRICE', 'B-MIN_PRICE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MAX_PRICE', 'B-MAX_PRICE', 'B-MAX_PRICE', 'B-MAX_PRICE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'

In [ ]:
# Create a mapping from string labels to unique integers
label_map = {label: i for i, label in enumerate(ner_classes)}
print(label_map)

{'B-PRODUCT_TYPE': 0, 'I-PRODUCT_TYPE': 1, 'B-SKIN_CONCERN': 2, 'B-PREF_INGREDIENT': 3, 'I-PREF_INGREDIENT': 4, 'B-MIN_PRICE': 5, 'B-MAX_PRICE': 6, 'B-EXCL_INGREDIENT': 7, 'I-EXCL_INGREDIENT': 8, 'O': 9}


In [ ]:
# Convert the corrected labels to numeric indices
numeric_labels = convert_labels_to_indices(corrected_labels, label_map)

In [ ]:
# Convert data to tensors and create a DataLoader
input_ids, attention_mask, _ = zip(*processed_data)
input_ids_tensor = torch.tensor(input_ids, dtype=torch.long)
attention_mask_tensor = torch.tensor(attention_mask, dtype=torch.long)
labels_tensor = torch.tensor(numeric_labels, dtype=torch.long)

dataset = TensorDataset(input_ids_tensor, attention_mask_tensor, labels_tensor)
dataloader = DataLoader(dataset, batch_size=32)

## Training with BERT

Here's where the fun stuff happens

In [ ]:
from transformers import DistilBertForTokenClassification, AdamW

# Define the number of epochs
epochs = 100  # adjust as needed later

# Define the model for DistilBERT
model = DistilBertForTokenClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=len(label_map),  # Number of distinct labels in dataset
    output_attentions=False,
    output_hidden_states=False
)


In [ ]:
# Set training params
optimizer = AdamW(model.parameters(), lr=1e-5)  # can adjust the learning rate as we train...


In [ ]:
#set to use GPU when we can
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


Set-up WandB before training

In [ ]:
# Initialize a new run
run = wandb.init(project="DLAIE Project F23")

# Track hyperparameters and run metadata
wandb.config = {
  "learning_rate": 1e-5,
  "epochs": 100,
  "batch_size": 32
}

In [ ]:
#training loop
#model is learning from our data
#iterate over the DataLoader, passing data to model
#update model's weights based on the output

# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        # Forward pass
        outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        total_loss += loss.item()

        # Log batch loss to W&B
        wandb.log({"batch_loss": loss.item()})

        # Backward pass and update
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    # Calculate the average loss over the epoch
    avg_train_loss = total_loss / len(dataloader)

    # Log average training loss to W&B
    wandb.log({"epoch": epoch, "avg_train_loss": avg_train_loss})

    print(f"Average training loss for epoch {epoch}: {avg_train_loss}")

In [ ]:
# this cell saves the model we just trained to a google drive folder
# folder must be set up and named acordingly ahead of time!

# this allows us to call that specific model later and use it for validation, evaluation, etc.

# setting this equal to wandb's name for the run
run_name = run.name

from google.colab import drive
drive.mount('/content/drive')

# Create a directory name with the W&B run name in Google Drive
model_save_path = f'/content/drive/My Drive/DLAIEF23_Models/{run_name}'

# Save the model to this directory
model.save_pretrained(model_save_path)

In [ ]:
# Finish the W&B run
wandb.finish()

# Validation
val_df

NER section MUST run before for this to work.

Can skip training section as written.

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForTokenClassification

# calling our saved model
model_path = '/content/drive/MyDrive/DLAIEF23_Models/dulcet-aardvark-32'

# Initialize the tokenizer
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

# Import our model
model = DistilBertForTokenClassification.from_pretrained(model_path)


In [ ]:
# call process_annotation function from earlier for validation data
processed_validation_data = [process_annotation(annotation) for annotation in val_df if annotation is not None]

# Print the number of None values discarded
print(f"Number of None values discarded: {none_counter}")

Number of None values discarded: 0


In [ ]:
#double check we've dropped our none's
for item in processed_validation_data:
    if item is None or not isinstance(item, tuple) or len(item) != 3:
        print("Found an issue with processed data:", item)


In [ ]:
# Extract labels from processed val data
extracted_val_labels = [aligned_labels for _, _, aligned_labels in processed_validation_data if aligned_labels is not None]
print(extracted_val_labels)

[['B-B-SKIN_CONCERN', 'I-B-SKIN_CONCERN', 'I-B-SKIN_CONCERN', 'I-B-SKIN_CONCERN', 'I-B-SKIN_CONCERN', 'I-B-SKIN_CONCERN', 'I-B-SKIN_CONCERN', 'I-B-SKIN_CONCERN', 'I-B-SKIN_CONCERN', 'I-B-SKIN_CONCERN', 'I-B-SKIN_CONCERN', 'I-B-SKIN_CONCERN', 'B-O', 'I-O', 'I-O', 'I-O', 'I-O', 'B-B-PRODUCT_TYPE', 'I-B-PRODUCT_TYPE', 'I-B-PRODUCT_TYPE', 'I-B-PRODUCT_TYPE', 'B-I-PRODUCT_TYPE', 'I-I-PRODUCT_TYPE', 'I-I-PRODUCT_TYPE', 'I-I-PRODUCT_TYPE', 'I-I-PRODUCT_TYPE', 'I-I-PRODUCT_TYPE', 'B-O', 'I-O', 'I-O', 'I-O', 'I-O', 'B-O', 'I-O', 'I-O', 'I-O', 'I-O', 'B-B-MIN_PRICE', 'I-B-MIN_PRICE', 'I-B-MIN_PRICE', 'B-O', 'I-O', 'I-O', 'I-O', 'B-O', 'I-O', 'I-O', 'I-O', 'I-O', 'B-O', 'I-O', 'I-O', 'I-O', 'I-O', 'B-B-MAX_PRICE', 'I-B-MAX_PRICE', 'I-B-MAX_PRICE', 'I-B-MAX_PRICE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

In [ ]:
# Correct the labels
corrected_val_labels = correct_labels(extracted_val_labels)
print(corrected_val_labels)

[['B-SKIN_CONCERN', 'B-SKIN_CONCERN', 'B-SKIN_CONCERN', 'B-SKIN_CONCERN', 'B-SKIN_CONCERN', 'B-SKIN_CONCERN', 'B-SKIN_CONCERN', 'B-SKIN_CONCERN', 'B-SKIN_CONCERN', 'B-SKIN_CONCERN', 'B-SKIN_CONCERN', 'B-SKIN_CONCERN', 'O', 'O', 'O', 'O', 'O', 'B-PRODUCT_TYPE', 'B-PRODUCT_TYPE', 'B-PRODUCT_TYPE', 'B-PRODUCT_TYPE', 'I-PRODUCT_TYPE', 'I-PRODUCT_TYPE', 'I-PRODUCT_TYPE', 'I-PRODUCT_TYPE', 'I-PRODUCT_TYPE', 'I-PRODUCT_TYPE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MIN_PRICE', 'B-MIN_PRICE', 'B-MIN_PRICE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MAX_PRICE', 'B-MAX_PRICE', 'B-MAX_PRICE', 'B-MAX_PRICE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O

In [ ]:
# this uses same label_map as training to ensure consistency
# Convert the corrected labels to numeric indices
numeric_val_labels = convert_labels_to_indices(corrected_val_labels, label_map)

In [ ]:
# Convert validation data to tensors
val_input_ids, val_attention_mask, _ = zip(*processed_validation_data)
val_input_ids_tensor = torch.tensor(val_input_ids, dtype=torch.long)
val_attention_mask_tensor = torch.tensor(val_attention_mask, dtype=torch.long)
val_labels_tensor = torch.tensor(numeric_val_labels, dtype=torch.long)

# Create a DataLoader for the validation data
validation_dataset = TensorDataset(val_input_ids_tensor, val_attention_mask_tensor, val_labels_tensor)
validation_dataloader = DataLoader(validation_dataset, batch_size=32)

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
import torch

# Set the model to use the GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Put model in evaluation mode
model.eval()

# Disable gradient calculations
with torch.no_grad():
    # Initialize lists to store true labels and predictions
    true_labels = []
    predictions = []

    # Iterate over batches in the validation dataloader
    for batch in validation_dataloader:  # Use validation_dataloader here
        # Unpack the batch data and move it to the GPU
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        # Forward pass, get model predictions
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = labels.to('cpu').numpy()

        # Convert logits to actual predictions
        batch_predictions = np.argmax(logits, axis=2)

        # Append batch predictions and true labels
        true_labels.append(label_ids)
        predictions.append(batch_predictions)


In [ ]:
# Flatten the lists of true labels and predictions
true_labels_flat = [label for batch in true_labels for label in batch.flatten()]
predictions_flat = [prediction for batch in predictions for prediction in batch.flatten()]

In [ ]:
# Filtering out 'ignore_index' (-100 for DistilBERT which is HuggingFace)
true_labels_filtered = [label for label in true_labels_flat if label != -100]
predictions_filtered = [pred for pred, label in zip(predictions_flat, true_labels_flat) if label != -100]


In [ ]:
# Calculate metrics
accuracy = accuracy_score(true_labels_filtered, predictions_filtered)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels_filtered, predictions_filtered, average='weighted')


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(f"Validation Accuracy: {accuracy}")
print(f"Validation Precision: {precision}")
print(f"Validation Recall: {recall}")
print(f"Validation F1-Score: {f1}")

Validation Accuracy: 0.9844618055555555
Validation Precision: 0.985064601644672
Validation Recall: 0.9844618055555555
Validation F1-Score: 0.9845083532090857


# Evaluation

NER section MUST run before for this to work.

Can skip training & validation sections as written.

## Set Up

Call our favorite model, make sure it's in evaluation mode

In [ ]:
from transformers import DistilBertForTokenClassification, DistilBertTokenizer

# Calling our model here so we can run this without running traning and validation cells
model_path = '/content/drive/MyDrive/DLAIEF23_Models/dulcet-aardvark-32'

# Load the tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForTokenClassification.from_pretrained(model_path)

# Ensure the model is in evaluation mode
model.eval()


DistilBertForTokenClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
    

In [ ]:
# Example user input - using this to test the functions I'm writing
user_input = "I'm looking for an eye cream with retinol"


In [ ]:
# Invert the label_map to map indices to labels
index_to_label_map = {index: label for label, index in label_map.items()}


In [ ]:
# Tokenize and run the model
inputs = tokenizer(user_input, return_tensors="pt", truncation=True, padding=True, max_length=512)
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=2)

# Convert predictions to labels
predicted_indices = predictions[0].tolist()
predicted_labels = [index_to_label_map[idx] for idx in predicted_indices]

# Tokenize the input text to match with predictions
tokenized_input = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

In [ ]:
entities = {}
current_entity = ""
current_type = None
for token, label in zip(tokenized_input, predicted_labels):
    if label.startswith("B-"):
        if current_entity and current_type:
            entities.setdefault(current_type, []).append(current_entity)
        current_entity = token.lstrip("##")  # Start a new entity
        current_type = label.split("-")[1]
    elif label.startswith("I-") and current_type == label.split("-")[1]:
        current_entity += " " + token.lstrip("##")  # Append to the current entity
    else:
        if current_entity and current_type:
            entities.setdefault(current_type, []).append(current_entity)
        current_entity = ""
        current_type = None

# Add the last entity if it exists
if current_entity and current_type:
    entities.setdefault(current_type, []).append(current_entity)

print("Extracted entities:", entities)
for token, label in zip(tokenized_input, predicted_labels):
    print(f"{token}: {label}")


Extracted entities: {'PRODUCT_TYPE': ['tino', 'l', '[SEP]']}
[CLS]: O
i: O
': O
m: O
looking: O
for: O
an: O
eye: O
cream: O
with: O
re: O
##tino: B-PRODUCT_TYPE
##l: B-PRODUCT_TYPE
[SEP]: B-PRODUCT_TYPE


## Product Type

In [ ]:
def filter_by_product_type(df, product_types):
    """
    Filter the DataFrame based on product type, handling multi-word labels.

    Args:
    df (DataFrame): The DataFrame to filter (sephora_df).
    product_types (list): List of product types extracted from the NER model.

    Returns:
    DataFrame: Filtered DataFrame.
    """
    try:
        # Normalize product types for case-insensitive comparison
        normalized_product_types = {ptype.replace(" ", "").lower() for ptype in product_types}

        # Function to check if the label matches any product type
        def label_matches(label):
            normalized_label = label.replace(" ", "").lower()
            return normalized_label in normalized_product_types

        # Filter DataFrame
        filtered_df = df[df['Label'].apply(label_matches)]
        return filtered_df
    except Exception as e:
        print(f"Error in filter_by_product_type: {e}")
        return df  # Return the original DataFrame in case of error


## Preferred Ingredient

In [ ]:
def filter_by_pref_ingredient(df, pref_ingredients):
    """
    Filter the DataFrame based on preferred ingredients.

    Args:
    df (DataFrame): The DataFrame to filter (sephora_df).
    pref_ingredients (list): List of preferred ingredients extracted from the NER model.

    Returns:
    DataFrame: Filtered DataFrame.
    """
    try:
        # Normalize preferred ingredients for case-insensitive comparison
        normalized_pref_ingredients = {ingredient.lower() for ingredient in pref_ingredients}

        # Function to check if the ingredients list contains any preferred ingredient
        def contains_pref_ingredient(ingredient_list):
           # Normalize and split the ingredients list for comparison
           ingredients = set(ing.lower() for ing in ingredient_list.split(', '))
           return any(ing in ingredients for ing in normalized_pref_ingredients)

        # Filter DataFrame
        filtered_df = df[df['ingredients'].apply(contains_pref_ingredient)]
        return filtered_df
    except Exception as e:
        print(f"Error in filter_by_pref_ingredient: {e}")
        return df  # Return the original DataFrame in case of error


## Excluded Ingredients

In [ ]:
def filter_by_excl_ingredient(df, excl_ingredients):
    """
    Filter the DataFrame to exclude products with certain ingredients.

    Args:
    df (DataFrame): The DataFrame to filter (sephora_df).
    excl_ingredients (list): List of excluded ingredients extracted from the NER model.

    Returns:
    DataFrame: Filtered DataFrame.
    """
    try:
        # Normalize excluded ingredients for case-insensitive comparison
        normalized_excl_ingredients = {ing.lower() for ing in excl_ingredients}

        # Function to check if the ingredients list contains any excluded ingredient
        def contains_excl_ingredient(ingredient_list):
            # Normalize and split the ingredients list for comparison
            ingredients = set(ing.lower() for ing in ingredient_list.split(', '))
            return not any(ing in ingredients for ing in normalized_excl_ingredients)

        # Filter DataFrame
        filtered_df = df[df['ingredients'].apply(contains_excl_ingredient)]
        return filtered_df
    except Exception as e:
        print(f"Error in filter_by_excl_ingredient: {e}")
        return df  # Return the original DataFrame in case of error


## Min Price

In [ ]:
def filter_by_min_price(df, min_price):
    """
    Filter the DataFrame to include only products with a price greater than or equal to min_price.

    Args:
    df (DataFrame): The DataFrame to filter (sephora_df).
    min_price (float): The minimum price threshold.

    Returns:
    DataFrame: Filtered DataFrame.
    """
    try:
        # Convert price from string to float and compare
        filtered_df = df[df[' price '].replace('[\$,]', '', regex=True).astype(float) >= min_price]
        return filtered_df
    except Exception as e:
        print(f"Error in filter_by_min_price: {e}")
        return df

## Max Price

In [ ]:
def filter_by_max_price(df, max_price):
    """
    Filter the DataFrame to include only products with a price less than or equal to max_price.

    Args:
    df (DataFrame): The DataFrame to filter (sephora_df).
    max_price (float): The maximum price threshold.

    Returns:
    DataFrame: Filtered DataFrame.
    """
    try:
        # Convert price from string to float and compare
        filtered_df = df[df[' price '].replace('[\$,]', '', regex=True).astype(float) <= max_price]
        return filtered_df
    except Exception as e:
        print(f"Error in filter_by_max_price: {e}")
        return df

## Skin Concerns

In [ ]:
def filter_by_skin_concern(sephora_df, ingredient_mapping_df, skin_concerns):
    """
    Filter the DataFrame based on skin concerns.

    Args:
    sephora_df (DataFrame): The DataFrame of products (sephora_df).
    ingredient_mapping_df (DataFrame): The DataFrame mapping skin concerns to preferred and excluded ingredients.
    skin_concerns (list): List of skin concerns extracted from the NER model.

    Returns:
    DataFrame: Filtered DataFrame.
    """
    # Normalize skin concerns for case-insensitive comparison
    normalized_skin_concerns = {concern.lower() for concern in skin_concerns}

    # Filter ingredient_mapping_df for these concerns
    concern_filter = ingredient_mapping_df['Skin Concern'].str.lower().apply(lambda x: any(concern in x for concern in normalized_skin_concerns))
    filtered_ingredient_mapping = ingredient_mapping_df[concern_filter]

    # Aggregate preferred and excluded ingredients
    preferred_ingredients = set()
    excluded_ingredients = set()
    for _, row in filtered_ingredient_mapping.iterrows():
        preferred_ingredients.update(row['Preferred Ingredients'].split(', '))
        excluded_ingredients.update(row['Excluded Ingredients'].split(', '))

    # Apply filters on sephora_df
    if preferred_ingredients:
        sephora_df = filter_by_pref_ingredient(sephora_df, list(preferred_ingredients))
    if excluded_ingredients:
        sephora_df = filter_by_excl_ingredient(sephora_df, list(excluded_ingredients))

    return sephora_df


## Combining & Calling Filters

In [ ]:
# Filter by Product Type
if 'PRODUCT_TYPE' in entities:
    filtered_sephora_df = filter_by_product_type(sephora_df, entities['PRODUCT_TYPE'])

# Filter by Preferred Ingredient
if 'PREF_INGREDIENT' in entities:
    filtered_sephora_df = filter_by_pref_ingredient(filtered_sephora_df, entities['PREF_INGREDIENT'])

# Filter by Excluded Ingredient
excl_ingredients = []
if 'EXCL_INGREDIENT' in entities:
    for ing in entities['EXCL_INGREDIENT']:
        # Concatenate multi-word ingredients if they are split
        excl_ingredients.append(ing)
#filter the DataFrame
filtered_sephora_df = filter_by_excl_ingredient(filtered_sephora_df, excl_ingredients)

# Extraction of MIN_PRICE and MAX_PRICE
min_price = float(entities['MIN_PRICE'][0]) if 'MIN_PRICE' in entities else None
max_price = float(entities['MAX_PRICE'][0]) if 'MAX_PRICE' in entities else None

# Apply price filters
if min_price is not None:
    filtered_sephora_df = filter_by_min_price(filtered_sephora_df, min_price)

if max_price is not None:
    filtered_sephora_df = filter_by_max_price(filtered_sephora_df, max_price)

# Assuming 'entities' dictionary from model output
if 'SKIN_CONCERN' in entities:
    filtered_sephora_df = filter_by_skin_concern(filtered_sephora_df, ingredient_mapping_df, entities['SKIN_CONCERN'])

# Print the final filtered DataFrame
print("DataFrame after Skin Concern Filtering:", filtered_sephora_df)


DataFrame after Skin Concern Filtering: Empty DataFrame
Columns: []
Index: []


In [ ]:
# Manually set entity values for testing
test_entities = {
    'PRODUCT_TYPE': ['moisturizer'],
    'PREF_INGREDIENT': ['hyaluronic acid'],
    'EXCL_INGREDIENT': ['parabens'],
    'MIN_PRICE': ['20'],  # Representing as strings to simulate model output
    'MAX_PRICE': ['50'],
    'SKIN_CONCERN': ['dryness']
}

# Apply all filters based on test entities
filtered_sephora_df = sephora_df.copy()  # Start with the original DataFrame

if 'PRODUCT_TYPE' in test_entities:
    filtered_sephora_df = filter_by_product_type(filtered_sephora_df, test_entities['PRODUCT_TYPE'])

if 'PREF_INGREDIENT' in test_entities:
    filtered_sephora_df = filter_by_pref_ingredient(filtered_sephora_df, test_entities['PREF_INGREDIENT'])

if 'EXCL_INGREDIENT' in test_entities:
    excl_ingredients = test_entities['EXCL_INGREDIENT']
    filtered_sephora_df = filter_by_excl_ingredient(filtered_sephora_df, excl_ingredients)

if 'MIN_PRICE' in test_entities:
    min_price = float(test_entities['MIN_PRICE'][0])
    filtered_sephora_df = filter_by_min_price(filtered_sephora_df, min_price)

if 'MAX_PRICE' in test_entities:
    max_price = float(test_entities['MAX_PRICE'][0])
    filtered_sephora_df = filter_by_max_price(filtered_sephora_df, max_price)

if 'SKIN_CONCERN' in test_entities:
    filtered_sephora_df = filter_by_skin_concern(filtered_sephora_df, ingredient_mapping_df, test_entities['SKIN_CONCERN'])

# Print the final filtered DataFrame
print("Final Filtered DataFrame after all tests:", filtered_sephora_df)


Final Filtered DataFrame after all tests:             Label               brand  \
469   Moisturizer    FIRST AID BEAUTY   
597   Moisturizer        IT COSMETICS   
598   Moisturizer        IT COSMETICS   
888   Moisturizer        OLEHENRIKSEN   
1039  Moisturizer          PHILOSOPHY   
1087  Moisturizer  SEPHORA COLLECTION   
1244  Moisturizer               TARTE   

                                                   name    price   rank  \
469                       Ultra Repair® Hydrating Serum   $36.00    4.2   
597   Your Skin But Better™ CC+ Airbrush Perfecting ...   $35.00    3.6   
598      Bye Bye Redness™ Neutralizing Correcting Cream   $32.00    4.2   
888                       C-Rush™ Brightening Gel Crème   $44.00    4.6   
1039  Renewed Hope in A Jar Refreshing & Refining Mo...   $39.00    4.1   
1087              Intensive Instant Moisturizer + Cream   $23.00    4.3   
1244  Drink of H2O Hydrating Boost Moisturizer Rainf...   $39.00    4.4   

                            

In [ ]:
# Print the first few entries of the price column
print(sephora_df[' price '].head())


0     $112.00 
1      $94.00 
2      $28.00 
3      $95.00 
4      $98.00 
Name:  price , dtype: object


# Gradio Example

In [1]:
#for demonstration purposes and not based on our project, just an idea of what we were trying to achieve
def sample_gradio(user_input):
  if "moisturizer" in user_input:
    return "BOBBI BROWN,Hydrating Face Cream Moisturizer, $60.00"
  if "treatment" in user_input:
    return "CAUDALIE,Vine[Activ] Vitamin C Anti-Wrinkle Serum, $52.00"
  if "sunscreen" in user_input:
    return "SUPERGOOP!,Unseen Sunscreen Broad Spectrum SPF 40, $32.00"
  if "eye cream" in user_input:
    return "FRESH,Black Tea Firming and De-Puffing Eye Cream, $68.00"
  if "face mask" in user_input:
    return "GLAMGLOW,BUBBLESHEET™ Oxygenating Deep Cleanse Mask"
  else:
    return "no product match found"

In [7]:
# Define the Gradio interface
iface = gradio.Interface(
    fn=sample_gradio,
    inputs="text",
    outputs="text"
)

# Launch the Gradio interface
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e0dbadedcd0cbb21d9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
